# **Challenge: Validating a linear regression**

In [11]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import cross_validation
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [12]:
df = pd.read_csv('ny_crime.csv')

df = df.fillna(0)

df.columns = ['index', 'city', 'population', 'violent_crime', 'murder', 'rape_1', 'rape_2', 'robbery', 'aggravated_assault', 'property_crime', 'burglary', 'larceny-theft', 'motor_vehicle_theft', 'arson', 'population_sq', 'murder_cat', 'robbery_cat']

df = df.drop('index', axis=1)

print(df.dtypes)

df.head()

city                    object
population             float64
violent_crime           object
murder                 float64
rape_1                 float64
rape_2                  object
robbery                float64
aggravated_assault      object
property_crime         float64
burglary                object
larceny-theft           object
motor_vehicle_theft     object
arson                  float64
population_sq          float64
murder_cat               int64
robbery_cat              int64
dtype: object


,city,population,violent_crime,murder,rape_1,rape_2,robbery,aggravated_assault,property_crime,burglary,larceny-theft,motor_vehicle_theft,arson,population_sq,murder_cat,robbery_cat
0,Adams Village,1861.000,0,0.000,0.000,0,0.000,0,12.000,2,10,0,0.000,3463321.000,0,0
1,Addison Town and Village,2577.000,3,0.000,0.000,0,0.000,3,24.000,3,20,1,0.000,6640929.000,0,0
2,Akron Village,2846.000,3,0.000,0.000,0,0.000,3,16.000,1,15,0,0.000,8099716.000,0,0
3,Albany,97956.000,791,8.000,0.000,30,227.000,526,4090.000,705,"3,243",142,0.000,9595377936.000,1,1
4,Albion Village,6388.000,23,0.000,0.000,3,4.000,16,223.000,53,165,5,0.000,40806544.000,0,1


In [13]:
data = df[['population', 'population_sq', 'murder_cat', 'robbery_cat', 'property_crime']].dropna()
data.head()

,population,population_sq,murder_cat,robbery_cat,property_crime
0,1861.000,3463321.000,0,0,12.000
1,2577.000,6640929.000,0,0,24.000
2,2846.000,8099716.000,0,0,16.000
3,97956.000,9595377936.000,1,1,4090.000
4,6388.000,40806544.000,0,1,223.000


# **Initial Model: Simple Linear Regression**

In [14]:
regr = linear_model.LinearRegression()
X = df[['population', 'population_sq', 'murder_cat', 'robbery_cat']]
y = df['property_crime']
regr.fit(X, y)

print('\nCoefficients:\n', regr.coef_)
print('\nIntercept:\n', regr.intercept_)
print('\nR-squared:\n', regr.score(X, y))


Coefficients:
 [  1.40081564e-02   1.13436742e-07   8.42743726e+01   9.99603923e+01]

Intercept:
 -20.6331275019

R-squared:
 0.842167048595


# **Cross Validation**

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

scores = cross_val_score(regr, X_train, y_train, cv=3)

print('Accuracy:\n', regr.score(X_test, y_test))
print('\nCross Validation Scores:\n', regr.coef_)
print('\nMean:\n', regr.intercept_)
print('\nStandard Deviation:\n', regr.score(X, y))
print('\nRMSE:\n', np.sqrt(mean_squared_error(y_test, y_pred)))

Accuracy:
 0.74814561224

Cross Validation Scores:
 [  1.31341133e-02   1.16683085e-07   3.91563824e+01   1.07077750e+02]

Mean:
 -14.5082632531

Standard Deviation:
 0.841661122549

RMSE:
 310.793190468


# **Ordinary Least Squares Regression**

## **Iteration 1**

In [16]:
linear_formula = 'property_crime ~ population+population_sq+murder_cat+robbery_cat'

lm = smf.ols(formula=linear_formula, data=data).fit()

print('Parameters:\n', lm.params)
print('\nP-values:\n', lm.pvalues)
print('\nR-squared:\n', lm.rsquared)
print('\nConfidence Interval:')
lm.conf_int()

Parameters:
 Intercept       -20.633
population        0.014
population_sq     0.000
murder_cat       84.274
robbery_cat      99.960
dtype: float64

P-values:
 Intercept       0.569
population      0.000
population_sq   0.000
murder_cat      0.272
robbery_cat     0.054
dtype: float64

R-squared:
 0.842167048595

Confidence Interval:


,0,1
Intercept,-91.739,50.473
population,0.010,0.018
population_sq,0.000,0.000
murder_cat,-66.512,235.060
robbery_cat,-1.600,201.521


## **Iteration 2**

In [17]:
linear_formula = 'property_crime ~ population+murder_cat+robbery_cat'

lm = smf.ols(formula=linear_formula, data=data).fit()

print('Parameters:\n', lm.params)
print('\nP-values:\n', lm.pvalues)
print('\nR-squared:\n', lm.rsquared)
print('\nConfidence Interval:\n')
lm.conf_int()

Parameters:
 Intercept     -109.887
population       0.035
murder_cat    -192.262
robbery_cat    -67.261
dtype: float64

P-values:
 Intercept     0.007
population    0.000
murder_cat    0.020
robbery_cat   0.232
dtype: float64

R-squared:
 0.792267884045

Confidence Interval:



,0,1
Intercept,-189.021,-30.754
population,0.033,0.037
murder_cat,-354.301,-30.223
robbery_cat,-177.835,43.313


## **Iteration 3**

In [18]:
linear_formula = 'property_crime ~ population+robbery_cat'

lm = smf.ols(formula=linear_formula, data=data).fit()

print('Parameters:\n', lm.params)
print('\nP-values:\n', lm.pvalues)
print('\nR-squared:\n', lm.rsquared)
print('\nConfidence Interval:\n')
lm.conf_int()

Parameters:
 Intercept     -107.909
population       0.034
robbery_cat    -93.411
dtype: float64

P-values:
 Intercept     0.008
population    0.000
robbery_cat   0.093
dtype: float64

R-squared:
 0.788979001538

Confidence Interval:



,0,1
Intercept,-187.533,-28.286
population,0.032,0.036
robbery_cat,-202.461,15.639


# **Updated Model**

## **Remove population_sq feature**

In [19]:
regr = linear_model.LinearRegression()
X = df[['population', 'murder_cat', 'robbery_cat']]
y = df['property_crime']
regr.fit(X, y)

print('Intercept:\n', regr.coef_)
print('\nR-squared:\n', regr.coef_)
print('\nCoefficients:\n', regr.score(X, y))

Intercept:
 [  3.52053648e-02  -1.92261770e+02  -6.72605959e+01]

R-squared:
 [  3.52053648e-02  -1.92261770e+02  -6.72605959e+01]

Coefficients:
 0.792267884045


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

scores = cross_val_score(regr, X_train, y_train, cv=3)

print('Accuracy:\n', regr.score(X_test, y_test))
print('\nCross Validation Scores:\n', regr.coef_)
print('\nMean:\n', regr.intercept_)
print('\nStandard Deviation:\n', regr.score(X, y))
print('\nRMSE:\n', np.sqrt(mean_squared_error(y_test, y_pred)))

Accuracy:
 0.648811053746

Cross Validation Scores:
 [  3.61202828e-02  -2.85028829e+02  -7.28313204e+01]

Mean:
 -104.803925707

Standard Deviation:
 0.791285668056

RMSE:
 367.000920053


## **Remove population_sq and murder features**

In [21]:
regr = linear_model.LinearRegression()
X = df[['population', 'robbery_cat']]
y = df['property_crime']
regr.fit(X, y)

print('Intercept:\n', regr.coef_)
print('\nR-squared:\n', regr.coef_)
print('\nCoefficients:\n', regr.score(X, y))

Intercept:
 [  3.44661269e-02  -9.34108858e+01]

R-squared:
 [  3.44661269e-02  -9.34108858e+01]

Coefficients:
 0.788979001538


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

scores = cross_val_score(regr, X_train, y_train, cv=3)

print('Accuracy:\n', regr.score(X_test, y_test))
print('\nCross Validation Scores:\n', regr.coef_)
print('\nMean:\n', regr.intercept_)
print('\nStandard Deviation:\n', regr.score(X, y))
print('\nRMSE:\n', np.sqrt(mean_squared_error(y_test, y_pred)))

Accuracy:
 0.692031352078

Cross Validation Scores:
 [  3.51167136e-02  -1.10740237e+02]

Mean:
 -103.121658149

Standard Deviation:
 0.788696361995

RMSE:
 343.676618985


# **Validating Model with 2014 NY Crime Dataset**

In [23]:
df_2014 = pd.read_csv('Table_8_Offenses_Known_to_Law_Enforcement_by_New_York_by_City_2014.csv')

df_2014.columns = ['city', 'population', 'violent_crime', 'murder', 'rape_1', 'rape_2', 'robbery', 'aggravated_assault', 'property_crime', 'burglary', 'larceny-theft', 'motor_vehicle_theft', 'arson']

print(df_2014.dtypes)

df_2014.head()

city                    object
population              object
violent_crime           object
murder                 float64
rape_1                  object
rape_2                 float64
robbery                 object
aggravated_assault      object
property_crime          object
burglary                object
larceny-theft           object
motor_vehicle_theft     object
arson                  float64
dtype: object


,city,population,violent_crime,murder,rape_1,rape_2,robbery,aggravated_assault,property_crime,burglary,larceny-theft,motor_vehicle_theft,arson
0,Adams Village,"1,851",0,0.000,NaN,0.000,0,0,11,1,10,0,0.000
1,Addison Town and Village,"2,568",2,0.000,NaN,0.000,1,1,49,1,47,1,0.000
2,Afton Village4,820,0,0.000,0,nan,0,0,1,0,1,0,0.000
3,Akron Village,"2,842",1,0.000,NaN,0.000,0,1,17,0,17,0,0.000
4,Albany4,"98,595",802,8.000,54,nan,237,503,"3,888",683,"3,083",122,12.000


In [24]:
df_2014 = df_2014.fillna('0')

# Remove commas from numeric strings
df_2014['population'] = df_2014['population'].map(lambda x: int(x.replace(',', '')))

df_2014['murder'] = df_2014['murder'].map(lambda x: int(x))

df_2014['robbery'] = df_2014['robbery'].map(lambda x: int(x.replace(',', '')))

df_2014['property_crime'] = df_2014['property_crime'].map(lambda x: int(x.replace(',', '')))

print('df_2014.dtypes: ', df_2014.dtypes)

df.head()

df_2014.dtypes:  city                   object
population              int64
violent_crime          object
murder                  int64
rape_1                 object
rape_2                 object
robbery                 int64
aggravated_assault     object
property_crime          int64
burglary               object
larceny-theft          object
motor_vehicle_theft    object
arson                  object
dtype: object


,city,population,violent_crime,murder,rape_1,rape_2,robbery,aggravated_assault,property_crime,burglary,larceny-theft,motor_vehicle_theft,arson,population_sq,murder_cat,robbery_cat
0,Adams Village,1861.000,0,0.000,0.000,0,0.000,0,12.000,2,10,0,0.000,3463321.000,0,0
1,Addison Town and Village,2577.000,3,0.000,0.000,0,0.000,3,24.000,3,20,1,0.000,6640929.000,0,0
2,Akron Village,2846.000,3,0.000,0.000,0,0.000,3,16.000,1,15,0,0.000,8099716.000,0,0
3,Albany,97956.000,791,8.000,0.000,30,227.000,526,4090.000,705,"3,243",142,0.000,9595377936.000,1,1
4,Albion Village,6388.000,23,0.000,0.000,3,4.000,16,223.000,53,165,5,0.000,40806544.000,0,1


In [25]:
df_2014 = df_2014.fillna(0)

# Filter out any outliers over two standard deviations above the mean
pop_cutoff = df_2014['population'].mean() + 2*df_2014['population'].std()
mur_cutoff = df_2014['murder'].mean() + 2*df_2014['murder'].std()
rob_cutoff = df_2014['robbery'].mean() + 2*df_2014['robbery'].std()
prop_cutoff = df_2014['property_crime'].mean() + 2*df_2014['property_crime'].std()

df_2014['population'] = df_2014['population'].map(lambda x: x if x < pop_cutoff else None)
df_2014['murder'] = df_2014['murder'].map(lambda x: x if x < mur_cutoff else None)
df_2014['robbery'] = df_2014['robbery'].map(lambda x: x if x < rob_cutoff else None)
df_2014['property_crime'] = df_2014['property_crime'].map(lambda x: x if x < prop_cutoff else None)

df_2014.describe()

,population,murder,robbery,property_crime
count,370.000,369.000,370.000,370.000
mean,14810.086,0.388,15.684,327.703
std,26519.851,1.982,84.386,954.565
min,0.000,0.000,0.000,0.000
25%,2603.500,0.000,0.000,24.000
50%,6507.500,0.000,1.000,75.000
75%,15237.250,0.000,4.000,267.500
max,258419.000,27.000,1277.000,12449.000


In [26]:
# Create new feature
df_2014['population_sq'] = df_2014['population']**2

df_2014.head()

,city,population,violent_crime,murder,rape_1,rape_2,robbery,aggravated_assault,property_crime,burglary,larceny-theft,motor_vehicle_theft,arson,population_sq
0,Adams Village,1851.000,0,0.000,0,0.000,0.000,0,11.000,1,10,0,0.000,3426201.000
1,Addison Town and Village,2568.000,2,0.000,0,0.000,1.000,1,49.000,1,47,1,0.000,6594624.000
2,Afton Village4,820.000,0,0.000,0,0,0.000,0,1.000,0,1,0,0.000,672400.000
3,Akron Village,2842.000,1,0.000,0,0.000,0.000,1,17.000,0,17,0,0.000,8076964.000
4,Albany4,98595.000,802,8.000,54,0,237.000,503,3888.000,683,"3,083",122,12.000,9720974025.000


In [27]:
# Create categorical features
df_2014['murder_cat'] = df_2014['murder'].dropna().map(lambda x: 1 if x > 0 else 0)
df_2014['robbery_cat'] = df_2014['robbery'].dropna().map(lambda x: 1 if x > 0 else 0)

df_2014.head()

,city,population,violent_crime,murder,rape_1,rape_2,robbery,aggravated_assault,property_crime,burglary,larceny-theft,motor_vehicle_theft,arson,population_sq,murder_cat,robbery_cat
0,Adams Village,1851.000,0,0.000,0,0.000,0.000,0,11.000,1,10,0,0.000,3426201.000,0.000,0.000
1,Addison Town and Village,2568.000,2,0.000,0,0.000,1.000,1,49.000,1,47,1,0.000,6594624.000,0.000,1.000
2,Afton Village4,820.000,0,0.000,0,0,0.000,0,1.000,0,1,0,0.000,672400.000,0.000,0.000
3,Akron Village,2842.000,1,0.000,0,0.000,0.000,1,17.000,0,17,0,0.000,8076964.000,0.000,0.000
4,Albany4,98595.000,802,8.000,54,0,237.000,503,3888.000,683,"3,083",122,12.000,9720974025.000,1.000,1.000


In [28]:
data = df_2014[['population', 'robbery_cat', 'property_crime']].dropna()

linear_formula = 'property_crime ~ population+robbery_cat'

lm = smf.ols(formula=linear_formula, data=data).fit()

y_pred_2014 = lm.predict(data[['population', 'robbery_cat']])

print('RMSE (2014 Data): ', np.sqrt(mean_squared_error(data['property_crime'], y_pred_2014)))

RMSE (2014 Data):  447.451763791
